In [ ]:
import h5pyd as h5py

In [ ]:
print(h5py.__version__)

In [ ]:
! hsls -H -v /home/jreadey/ESIP/

## Now try reading with xarray (using h5netcdf + h5pyd)

In [ ]:
import xarray as xr

In [ ]:
url_hsds = "http://hsdshdflab.hdfgroup.org/home/jreadey/ESIP/adcirc_native.nc"

In [ ]:
%%time
ds_hsds = xr.open_dataset(url_hsds, engine='h5netcdf', chunks={'time':10, 'node':141973})

In [ ]:
ds_hsds['zeta']

## xarray with local Dask cluster (since my docker container doesn't have h5pyd)

In [ ]:
from dask.distributed import Client, progress, LocalCluster, progress

In [ ]:
cluster = LocalCluster()

In [ ]:
client = Client(cluster)

In [ ]:
client

In [ ]:
ds_hsds1 = xr.open_dataset(url_hsds, engine='h5netcdf', chunks={'time':10, 'node':141973})

In [ ]:
ds_hsds1['zeta']

In [ ]:
%%time
max_var = ds_hsds1['zeta'].max(dim='time').compute()

In [ ]:
4*60+14